<a href="https://colab.research.google.com/github/Anakha-s/Project-1/blob/main/eg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## EDA Pkgs
import pandas as pd
#Data Viz Pkg
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import numpy as np # linear algebra
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer,PorterStemmer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Load Dataset
df = pd.read_csv("covid19_tweets.csv")
# Preview
df.head(3)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False


In [ ]:
!pip install neattext
# Load Text Cleaning Package
import neattext.functions as nfx

     |████████████████████████████████| 114 kB 4.3 MB/s 


In [ ]:
# Cleaning Text
df['clean_tweet'] = df['text'].apply(nfx.remove_hashtags)
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: nfx.remove_userhandles(x))
# Cleaning Text: Multiple WhiteSpaces
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_multiple_spaces)
# Cleaning Text : Remove urls
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_urls)
# Cleaning Text: Punctuations
df['clean_tweet'] = df['clean_tweet'].apply(nfx.remove_puncts)
#Remove blank rows if any.
df['clean_tweet'] .dropna(inplace=True)
# Change all the text to lower case.
df['clean_tweet']  = [entry.lower() for entry in df['clean_tweet'] ]
df[['text','clean_tweet']]

,text,clean_tweet
0,If I smelled the scent of hand sanitizers toda...,if i smelled the scent of hand sanitizers toda...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,hey and wouldnt it have made more sense to ha...
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,trump never once claimed was a hoax we all cl...
3,@brookbanktv The one gift #COVID19 has give me...,the one gift has give me is an appreciation f...
4,25 July : Media Bulletin on Novel #CoronaVirus...,25 july : media bulletin on novel
...,...,...
179103,Thanks @IamOhmai for nominating me for the @WH...,thanks for nominating me for the challenge i n...
179104,2020! The year of insanity! Lol! #COVID19 http...,2020 the year of insanity lol
179105,@CTVNews A powerful painting by Juan Lucena. I...,a powerful painting by juan lucena its a trib...
179106,"More than 1,200 students test positive for #CO...",more than 1200 students test positive for at m...


In [ ]:
# Tokenization
df['clean_tweet']= [word_tokenize(entry) for entry in df['clean_tweet']]
df['clean_tweet']
.


In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(words):
  return [word for word in words if  not word in stop_words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: remove_stopwords(x))


#def stemming(words):
  #ps=PorterStemmer()
  #return [ps.stem(word) for word in words]
#df['clean_tweet']=df['clean_tweet'].apply(lambda x: stemming(x))

def lemmatizing(words):
    lemmatizer =WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]
df['clean_tweet']=df['clean_tweet'].apply(lambda x: lemmatizing(x))

def final_text(words):
     return ' '.join(words)
df['clean_tweet']=df['clean_tweet'].apply(lambda x:final_text(x))
   


In [ ]:
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment_polarity = blob.sentiment.polarity
    sentiment_subjectivity = blob.sentiment.subjectivity
    if sentiment_polarity > 0:
        sentiment_label = 'Positive'
    elif sentiment_polarity < 0:
        sentiment_label = 'Negative'
    else:
        sentiment_label = 'Neutral'
    result = {'polarity':sentiment_polarity,
              'subjectivity':sentiment_subjectivity,
              'sentiment':sentiment_label}
    return result

In [ ]:
df['sentiment_results'] = df['clean_tweet'].apply(get_sentiment)
df['sentiment_results']

0         {'polarity': -0.25, 'subjectivity': 0.25, 'sen...
1         {'polarity': 0.5, 'subjectivity': 0.5, 'sentim...
2         {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
3         {'polarity': 0.0, 'subjectivity': 0.3571428571...
4         {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
                                ...                        
179103    {'polarity': 0.2, 'subjectivity': 0.2, 'sentim...
179104    {'polarity': 0.8, 'subjectivity': 0.7, 'sentim...
179105    {'polarity': 0.3, 'subjectivity': 1.0, 'sentim...
179106    {'polarity': 0.26325757575757575, 'subjectivit...
179107    {'polarity': 0.0, 'subjectivity': 0.0, 'sentim...
Name: sentiment_results, Length: 179108, dtype: object

In [ ]:
df = df.join(pd.json_normalize(df['sentiment_results']))
df.head(3)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,clean_tweet,sentiment_results,polarity,subjectivity,sentiment
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False,if i smelled the scent of hand sanitizers toda...,"{'polarity': -0.25, 'subjectivity': 0.25, 'sen...",-0.25,0.25,Negative
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False,hey and wouldnt it have made more sense to ha...,"{'polarity': 0.5, 'subjectivity': 0.5, 'sentim...",0.50,0.50,Positive
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False,trump never once claimed was a hoax we all cl...,"{'polarity': 0.0, 'subjectivity': 0.0, 'sentim...",0.00,0.00,Neutral


0         [if, i, smelled, the, scent, of, hand, sanitiz...
1         [hey, and, wouldnt, it, have, made, more, sens...
2         [trump, never, once, claimed, was, a, hoax, we...
3         [the, one, gift, has, give, me, is, an, apprec...
4                 [25, july, :, media, bulletin, on, novel]
                                ...                        
179103    [thanks, for, nominating, me, for, the, challe...
179104                 [2020, the, year, of, insanity, lol]
179105    [a, powerful, painting, by, juan, lucena, its,...
179106    [more, than, 1200, students, test, positive, f...
179107                     [i, stop, when, i, see, a, stop]
Name: clean_tweet, Length: 179108, dtype: object

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['clean_tweet'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

print(Tfidf_vect.vocabulary_)

{'smell': 4126, 'hand': 2191, 'sanit': 3915, 'today': 4532, 'someon': 4158, 'past': 3300, 'would': 4944, 'think': 4487, 'that': 4462, 'hey': 2257, 'wouldnt': 4945, 'made': 2799, 'sen': 3988, 'player': 3401, 'pay': 3315, 'respect': 3796, 'trump': 4617, 'never': 3085, 'claim': 1050, 'hoax': 2288, 'effort': 1654, 'to': 4531, 'one': 3189, 'gift': 2084, 'give': 2087, 'appreci': 564, 'simpl': 4082, 'thing': 4486, 'alway': 487, 'around': 587, 'me': 2870, '25': 162, 'juli': 2583, 'medium': 2885, 'bulletin': 875, 'novel': 3136, 'death': 1373, 'continu': 1203, 'rise': 3842, 'almost': 478, 'bad': 675, 'ever': 1767, 'politician': 3429, 'busi': 885, 'want': 4814, 'chang': 993, 'work': 4928, 'gener': 2070, 'recruit': 3702, 'specif': 4188, 'via': 4758, 'wear': 4835, 'face': 1842, 'cover': 1264, 'shop': 4047, 'includ': 2415, 'visit': 4789, 'local': 2754, 'commun': 1121, 'pray': 3475, 'good': 2110, 'health': 2230, 'recoveri': 3698, 'god': 2100, 'watch': 4825, 'covid19': 1270, 'case': 945, 'still': 4259

In [ ]:
print(Train_X_Tfidf)

  (0, 4316)	0.38899035620230105
  (0, 4239)	0.27692847231341056
  (0, 3503)	0.41779943105926753
  (0, 1721)	0.45478050277443643
  (0, 1493)	0.4070380467566097
  (0, 849)	0.4225905403925393
  (0, 504)	0.21529681226125108
  (1, 3946)	0.40506977254712245
  (1, 2886)	0.32899284901362597
  (1, 2295)	0.5448409439307244
  (1, 2118)	0.27187225184228475
  (1, 1939)	0.26654389782882043
  (1, 1511)	0.3135200860356329
  (1, 933)	0.277849652009644
  (1, 32)	0.3322274615710611
  (2, 4448)	0.22896209435566198
  (2, 3972)	0.29038841383878783
  (2, 3209)	0.5163417435758814
  (2, 2992)	0.35888679139763435
  (2, 2110)	0.29936600776421984
  (2, 2045)	0.3634612083708245
  (2, 2039)	0.3637421145952954
  (2, 1905)	0.3373574079999843
  (3, 4505)	0.3801098789032658
  (3, 4085)	0.2620235714869833
  :	:
  (125372, 3698)	0.22521320736653047
  (125372, 3028)	0.467178134663763
  (125372, 2563)	0.2408585157020062
  (125372, 2146)	0.3247237737316124
  (125372, 1733)	0.3757483421198125
  (125372, 1226)	0.3552286327135

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)